In [1]:
import numpy as np
import pandas as pd 
import bokeh.io
import bokeh.plotting
import scikit_posthocs as posthoc

bokeh.io.output_notebook()

Loading BokehJS ...

# Figure 2: CAP-Mac outperforms other engineered variants in newborn rhesus macaque in pool testing.
Load in the raw data file

In [2]:
df_infant_pool_raw = pd.read_csv("https://github.com/GradinaruLab/CAP-Mac/raw/main/raw-data/fig2.csv")
df_infant_pool_raw.head()

,variant,animal id,age,tissue id,xna,route,tissue,count
0,AAV9,RM-001,infant,RM-001_4,DNA,iv,brain,25653
1,AAV9,RM-001,infant,RM-001_8,DNA,iv,brain,33812
2,AAV9,RM-001,infant,RM-001_11,DNA,iv,brain,52424
3,AAV9,RM-001,infant,RM-001_4,RNA,iv,brain,4066
4,AAV9,RM-001,infant,RM-001_8,RNA,iv,brain,18210


Normalize the read count relative to the sum of all counts in each tissue sample. This includes the `virus` sample, which is the recovered viral genomes taken from the pooled virus used for injection for this particular experiment. 

In [3]:
df_infant_pool_raw["normalized count"] = df_infant_pool_raw.groupby(["animal id", "xna", "tissue", "tissue id"])["count"].transform(lambda x: x / x.sum())
enrich_grouped=df_infant_pool_raw.loc[df_infant_pool_raw["tissue"]=="virus"][["variant", "normalized count"]]
enrich_grouped.rename(columns={"normalized count": "virus normalized count"}, inplace=True)

df_infant_pool = pd.merge(df_infant_pool_raw, enrich_grouped,  on=["variant"])
df_infant_pool.head()

,variant,animal id,age,tissue id,xna,route,tissue,count,normalized count,virus normalized count
0,AAV9,RM-001,infant,RM-001_4,DNA,iv,brain,25653,0.103894,0.146356
1,AAV9,RM-001,infant,RM-001_8,DNA,iv,brain,33812,0.076046,0.146356
2,AAV9,RM-001,infant,RM-001_11,DNA,iv,brain,52424,0.078973,0.146356
3,AAV9,RM-001,infant,RM-001_4,RNA,iv,brain,4066,0.047577,0.146356
4,AAV9,RM-001,infant,RM-001_8,RNA,iv,brain,18210,0.054123,0.146356


Calculate enrichment as the ratio of `normalized count` over `virus normalized count`

In [4]:
df_infant_pool["enrichment"] = df_infant_pool["normalized count"] / df_infant_pool["virus normalized count"]
df_infant_pool.head()

,variant,animal id,age,tissue id,xna,route,tissue,count,normalized count,virus normalized count,enrichment
0,AAV9,RM-001,infant,RM-001_4,DNA,iv,brain,25653,0.103894,0.146356,0.709874
1,AAV9,RM-001,infant,RM-001_8,DNA,iv,brain,33812,0.076046,0.146356,0.519600
2,AAV9,RM-001,infant,RM-001_11,DNA,iv,brain,52424,0.078973,0.146356,0.539596
3,AAV9,RM-001,infant,RM-001_4,RNA,iv,brain,4066,0.047577,0.146356,0.325080
4,AAV9,RM-001,infant,RM-001_8,RNA,iv,brain,18210,0.054123,0.146356,0.369802


Express the enrichment as the fold-change relative to the mean AAV9 enrichment of the entire tissue for either DNA or RNA.

In [5]:
# Initialize empty DataFrame that we will populate
df_fold_change = pd.DataFrame()

for _, group in df_infant_pool.groupby(["tissue", "xna"]):
    mean_aav9_enrichment = group.loc[group["variant"]=="AAV9"]["enrichment"].mean()
    group["fold change"] = group["enrichment"] / mean_aav9_enrichment
    
    # df_delta = df_delta.append(group)
    df_fold_change = pd.concat([df_fold_change, group])

######## Make some changes for plotting neatness ############
# Specify columns as categorical and set order
df_fold_change["variant"] = pd.Categorical(df_fold_change["variant"], ["AAV9", "AAV-PHP.eB", "AAV.CAP-B2", "AAV.CAP-B10",
                                                                       "AAV.CAP-B22", "AAV9.452sub-LUNG1", "AAV.CAP-Mac", "AAV.CAP-C2"])
df_fold_change["animal id"] = pd.Categorical(df_fold_change["animal id"], ["RM-001", "RM-002", "virus"])
df_fold_change["xna"] = pd.Categorical(df_fold_change["xna"], ["DNA", "RNA"])

# Sort and reset index
df_fold_change = df_fold_change.sort_values(["variant", "animal id"]).reset_index(drop=True)
######## Make some changes for plotting neatness ############

df_fold_change.head()

,variant,animal id,age,tissue id,xna,route,tissue,count,normalized count,virus normalized count,enrichment,fold change
0,AAV9,RM-001,infant,RM-001_4,DNA,iv,brain,25653,0.103894,0.146356,0.709874,1.440737
1,AAV9,RM-001,infant,RM-001_8,DNA,iv,brain,33812,0.076046,0.146356,0.519600,1.054564
2,AAV9,RM-001,infant,RM-001_11,DNA,iv,brain,52424,0.078973,0.146356,0.539596,1.095147
3,AAV9,RM-001,infant,RM-001_4,RNA,iv,brain,4066,0.047577,0.146356,0.325080,0.820363
4,AAV9,RM-001,infant,RM-001_8,RNA,iv,brain,18210,0.054123,0.146356,0.369802,0.933223


Calculate the mean and standard error.

In [6]:
cols = ["variant", "animal id", "tissue", "tissue id", "xna", "fold change"]
df_fold_change = df_fold_change[(df_fold_change["animal id"]!="virus")][cols].reset_index(drop=True)


df_fold_change["cats"] = df_fold_change.apply(lambda x: (x["xna"], x["variant"]), axis=1)
df_stats = df_fold_change.groupby(["tissue", "variant", "xna"]).agg(mean=("fold change", "mean"), sem=("fold change", "sem"))
df_stats["mean fold change"] = df_stats["mean"]
df_stats["upper"] = df_stats["mean"] + df_stats["sem"]
df_stats["lower"] = df_stats["mean"] - df_stats["sem"]

df_plot = pd.merge(df_fold_change, df_stats[["mean fold change", "upper", "lower"]], on=["tissue", "variant", "xna"])
df_plot = df_plot.sort_values(by=["xna", "variant"])
df_plot.head()

,variant,animal id,tissue,tissue id,xna,fold change,cats,mean fold change,upper,lower
0,AAV9,RM-001,brain,RM-001_4,DNA,1.440737,"(DNA, AAV9)",1.0,1.105906,0.894094
1,AAV9,RM-001,brain,RM-001_8,DNA,1.054564,"(DNA, AAV9)",1.0,1.105906,0.894094
2,AAV9,RM-001,brain,RM-001_11,DNA,1.095147,"(DNA, AAV9)",1.0,1.105906,0.894094
3,AAV9,RM-002,brain,RM-002_4,DNA,0.765429,"(DNA, AAV9)",1.0,1.105906,0.894094
4,AAV9,RM-002,brain,RM-002_6,DNA,0.745074,"(DNA, AAV9)",1.0,1.105906,0.894094


## Figure 2b - Brain AAV barcode quantification

In [7]:
############### BOKEH FIGURE SETTINGS ###################
df_brain_plot = df_plot[df_plot["tissue"]=="brain"]
source_error = bokeh.models.ColumnDataSource(data=df_brain_plot)

x_range = list(df_brain_plot["cats"].unique())
animal_id = list(df_brain_plot["animal id"].unique())
marker_colors = list(bokeh.palettes.Colorblind[5])
markers = ["circle", "triangle", "square", "diamond", "plus"]

figure_width = 700
figure_height = 350
bar_width = .75
bar_fill_color="gray"
jitter_width = 0.3
marker_size = 10
bar_line_width = 1

error_size=10
error_line_width=.25
marker_line_width=1

p = bokeh.plotting.figure(x_range=bokeh.models.FactorRange(*x_range), title="Brain AAV barcode quantification (infant rhesus macaque; intravenous delivery)", 
                          height=figure_height, width=figure_width)

p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=np.pi/4
p.axis.major_tick_in = 0
p.axis.major_label_text_color="#000000"
p.axis.major_label_text_font_size="12pt"
p.xaxis.group_text_font_size="12pt"
p.xaxis.group_text_color="#000000"
p.axis.major_label_standoff=5
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [8]:
# Add bar and scatter plots
p.vbar(source=df_brain_plot[["cats", "mean fold change"]].drop_duplicates(), x="cats", top="mean fold change",
       width=bar_width, line_color="black", fill_color=bar_fill_color, line_width=bar_line_width)

p.scatter(source=df_brain_plot, x=bokeh.transform.jitter("cats", width=jitter_width, range=p.x_range), y="fold change", 
          marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_colors, animal_id), legend_group="animal id")

############### ADD ERROR BARS ###################
w = bokeh.models.Whisker(source=source_error, base="cats", upper="upper", lower="lower", level="overlay", line_width=error_line_width)

w.upper_head.line_width=error_line_width
w.upper_head.size=error_size
w.lower_head.line_width=error_line_width
w.lower_head.size=error_size
p.add_layout(w)
############### ADD ERROR BARS ###################

y_max = np.max([df_brain_plot["upper"], df_brain_plot["fold change"]])
y_upper = np.floor(y_max) - np.floor(y_max)%2 + 2
p.y_range = bokeh.models.Range1d(0, y_upper)
bokeh.io.show(p)

### Statistics for intravenous infant pool in brain

In [9]:
df_brain_dna = df_brain_plot.loc[df_brain_plot["xna"]=="DNA"]
p_val_brain_dna = posthoc.posthoc_tamhane(a=df_brain_dna, val_col="fold change", group_col="variant")

print("n = %i samples across %i animals. " % (len(df_brain_dna["tissue id"].unique()), len(df_brain_dna["animal id"].unique())))
print("AAV9 vs. AAV.CAP-Mac: P = %.2e." %(p_val_brain_dna.loc["AAV9", "AAV.CAP-Mac"]))
print("AAV9 vs. AAV.CAP-C2: P = %.2e." %(p_val_brain_dna.loc["AAV9", "AAV.CAP-C2"]))

n = 6 samples across 2 animals. 
AAV9 vs. AAV.CAP-Mac: P = 1.07e-04.
AAV9 vs. AAV.CAP-C2: P = 1.06e-06.


In [10]:
df_brain_rna = df_brain_plot.loc[df_brain_plot["xna"]=="RNA"]
p_val_brain_rna = posthoc.posthoc_tamhane(a=df_brain_rna, val_col="fold change", group_col="variant")

print("n = %i samples across %i animals. " % (len(df_brain_rna["tissue id"].unique()), len(df_brain_rna["animal id"].unique())))
print("AAV9 vs. AAV.CAP-Mac: P = %.4f." %(p_val_brain_rna.loc["AAV9", "AAV.CAP-Mac"]))
print("AAV9 vs. AAV.CAP-C2: P = %.5f." %(p_val_brain_rna.loc["AAV9", "AAV.CAP-C2"]))

n = 6 samples across 2 animals. 
AAV9 vs. AAV.CAP-Mac: P = 0.0388.
AAV9 vs. AAV.CAP-C2: P = 0.00190.


## Figure 2c - Liver AAV barcode quantification

In [11]:
############### BOKEH FIGURE SETTINGS ###################
df_liver_plot = df_plot[df_plot["tissue"]=="liver"]
source_error = bokeh.models.ColumnDataSource(data=df_liver_plot)

figure_width = 700
figure_height = 350
bar_width = .75
bar_fill_color="gray"
jitter_width = 0.3
marker_size = 10
bar_line_width = 1

error_size=10
error_line_width=.25
marker_line_width=1

p = bokeh.plotting.figure(x_range=bokeh.models.FactorRange(*x_range), title="Liver AAV barcode quantification (infant rhesus macaque; intravenous delivery)", 
                          height=figure_height, width=figure_width)

p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=np.pi/4
p.axis.major_tick_in = 0
p.axis.major_label_text_color="#000000"
p.axis.major_label_text_font_size="12pt"
p.xaxis.group_text_font_size="12pt"
p.xaxis.group_text_color="#000000"
p.axis.major_label_standoff=5
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [12]:
# Add bar and scatter plot
p.vbar(source=df_liver_plot[["cats", "mean fold change"]].drop_duplicates(), x="cats", top="mean fold change",
       width=bar_width, line_color="black", fill_color=bar_fill_color, line_width=bar_line_width)

p.scatter(source=df_liver_plot, x=bokeh.transform.jitter("cats", width=jitter_width, range=p.x_range), y="fold change", 
          marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_colors, animal_id), legend_group="animal id")

############### ADD ERROR BARS ###################
w = bokeh.models.Whisker(source=source_error, base="cats", upper="upper", lower="lower", level="overlay", line_width=error_line_width)

w.upper_head.line_width=error_line_width
w.upper_head.size=error_size
w.lower_head.line_width=error_line_width
w.lower_head.size=error_size
p.add_layout(w)
############### ADD ERROR BARS ###################

y_max = np.max([df_liver_plot["upper"], df_liver_plot["fold change"]])
y_upper = np.floor(y_max) - np.floor(y_max)%2 + 2

p.y_range = bokeh.models.Range1d(0, y_upper)
bokeh.io.show(p)